In [ ]:
!pip install qiskit numpy qiskit-aer qiskit[visualization] ipyvolume
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.circuit import Parameter
from numpy import empty, pi, arange, array
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [ ]:
theta = Parameter("$\\theta$")
g = Parameter("$\\gamma$")
b = Parameter("$\\beta$")

In [ ]:
# connection weights
w0 = 1
w1 = 1
w2 = 1
w3 = 1
w4 = 1
w5 = 1

# penalty for disconnected node
k = 3

In [ ]:
def CostFun(bit_string):
    i = 0
    c = empty([len(bit_string)], dtype=int)
    for char in bit_string[::-1]:
        c[i] = int(char)
        i = i + 1
    return ((9+4*k) + (w0-5-2*k)*c[0] + (w1-5-2*k)*c[1] + (w2-5-2*k)*c[2] + (w3-5-2*k)*c[3] + (w4-5-2*k)*c[4] + (w5-5-2*k)*c[5] +
            (2+k)*c[0]*c[1] + (2+k)*c[0]*c[2] + (2+k)*c[0]*c[3] + 2*c[0]*c[4] + (2+k)*c[0]*c[5] +
            (2+k)*c[1]*c[2] + (2+k)*c[1]*c[3] + (2+k)*c[1]*c[4] + 2*c[1]*c[5] +
            2*c[2]*c[3] + (2+k)*c[2]*c[4] + (2+k)*c[2]*c[5] +
            (2+k)*c[3]*c[4] + (2+k)*c[3]*c[5] + (2+k)*c[4]*c[5] -
            k*c[0]*c[1]*c[2] - k*c[0]*c[3]*c[5] - k*c[1]*c[3]*c[4] - k*c[2]*c[4]*c[5])

In [ ]:
def compute_expectation(counts):
    """Computes expectation value based on measurement results
    Args:
        counts: (dict) key as bit string, val as count
    Returns:
        avg: float
             expectation value
    """
    avg = 0
    sum_count = 0
    count_max = 0
    bit_string_max = ""
    for bit_string, count in counts.items():
        obj = CostFun(bit_string)
        avg += obj * count
        sum_count += count
        if (count > count_max):
            count_max = count
            bit_string_max = bit_string

    return avg/sum_count

In [ ]:
# We will also bring the different circuit components that
# build the qaoa circuit under a single function

def create_qaoa_circ(theta):
    """Creates a parametrized qaoa circuit
    Args:
        gamma,beta
        theta: (list) unitary parameters
    Returns:
        (QuantumCircuit) qiskit circuit
    """
    qc = QuantumCircuit(6)

    g = theta[0]
    b = theta[1]


    """
    g1 = theta[1]
    g2 = theta[2]

    g3 = theta[3]
    g4 = theta[4]
    g5 = theta[5]
    g6 = theta[6]
    g7 = theta[7]
    g8 = theta[8]
    g9 = theta[9]
    g10 = theta[10]
    g11 = theta[11]
    g12 = theta[12]
    g13 = theta[13]
    g14 = theta[14]
    g15 = theta[15]
    g16 = theta[16]
    g17 = theta[17]
    g18 = theta[18]
    g19 = theta[19]
    g20 = theta[20]
    g21 = theta[21]
    g22 = theta[22]
    g23 = theta[23]
    g24 = theta[24]

    """

    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    qc.h(4)
    qc.h(5)

    qc.rz((0.25*k-0.5*w0)*2*g,0)
    qc.rz((0.25*k-0.5*w1)*2*g,1)
    qc.rz((0.25*k-0.5*w2)*2*g,2)
    qc.rz((0.25*k-0.5*w3)*2*g,3)
    qc.rz((0.25*k-0.5*w4)*2*g,4)
    qc.rz((0.25*k-0.5*w5)*2*g,5)

    qc.rzz((0.5+0.125*k)*2*g,0,1)
    qc.rzz((0.5+0.125*k)*2*g,0,2)
    qc.rzz((0.5+0.125*k)*2*g,0,3)
    qc.rzz(0.5*2*g,0,4)
    qc.rzz((0.5+0.125*k)*2*g,0,5)

    qc.rzz((0.5+0.125*k)*2*g,1,2)
    qc.rzz((0.5+0.125*k)*2*g,1,3)
    qc.rzz((0.5+0.125*k)*2*g,1,4)
    qc.rzz(0.5*2*g,1,5)

    qc.rzz(0.5*2*g,2,3)
    qc.rzz((0.5+0.125*k)*2*g,2,4)
    qc.rzz((0.5+0.125*k)*2*g,2,5)

    qc.rzz((0.5+0.125*k)*2*g,3,4)
    qc.rzz((0.5+0.125*k)*2*g,3,5)

    qc.rzz((0.5+0.125*k)*2*g,4,5)

    qc.barrier()

    qc.cx(0,1)
    qc.rzz(k*0.125*2*g,1,2)
    qc.cx(0,1)

    qc.cx(0,3)
    qc.rzz(k*0.125*2*g,3,5)
    qc.cx(0,3)

    qc.cx(1,3)
    qc.rzz(k*0.125*2*g,3,4)
    qc.cx(1,3)

    qc.cx(2,4)
    qc.rzz(k*0.125*2*g,4,5)
    qc.cx(2,4)

    qc.barrier()

    qc.rx(2*b,0)
    qc.rx(2*b,1)
    qc.rx(2*b,2)
    qc.rx(2*b,3)
    qc.rx(2*b,4)
    qc.rx(2*b,5)

    qc.measure_all()

    return qc

In [ ]:
# Finally we write a function that executes the circuit
# on the chosen backend

def get_expectation():
    backend = AerSimulator()
    backend.shots = 10000

    def execute_circ(theta):
        print(theta[0]," ",theta[1])

        qc = create_qaoa_circ(theta)

        counts = backend.run(qc, seed_simulator=10,
                             shots=10000).result().get_counts()
        return compute_expectation(counts)

    return execute_circ

In [ ]:
# Run the whole thing

expectation = get_expectation()
res = minimize(expectation,
               [0.2, 2.8],
               method='COBYLA',
               tol=0.000000001,
               options={'disp': True})

print(res)

backend = AerSimulator()

qc_res = create_qaoa_circ(res.x)
counts = backend.run(qc_res, seed_simulator=10, shots=10000).result().get_counts()
plot_histogram(counts,figsize=(20,5))

In [ ]:
qc_res.draw(output='mpl',fold=-1)

In [ ]:
CostFun('000111')

In [ ]:
# This is gonna take some time... (cca 5 minutes)
# Preparation of 3D surface view of optimized ansatz expectation values for various gamma and beta

import ipyvolume as ipv
import numpy as np
X = arange(0,pi,pi/100)
Y = arange(0,pi,pi/100)
Xmesh, Ymesh = np.meshgrid(X, Y)
Zmesh = Xmesh * Ymesh
i=0
for gamma in X:
    j=0
    for beta in Y:
        theta = []
        theta.append(gamma)
        theta.append(beta)
        qc_res = create_qaoa_circ(theta)
        counts = backend.run(qc_res, seed_simulator=10, shots=10000).result().get_counts()
        e = compute_expectation(counts)
        Zmesh[j][i] = e
        j = j + 1
    i = i + 1


In [ ]:
# 3D surface view of optimized cost ansatz expectation values for various gamma and beta
# Use mouse wheel to zoom in/out

from matplotlib import cm
colormap = cm.coolwarm
znorm = Zmesh - Zmesh.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = colormap(znorm)

ipv.figure()
mesh = ipv.plot_surface(Xmesh, Ymesh, Zmesh, color=color[...,:3])
ipv.xlabel('Gamma')
ipv.xlim(0,pi)
ipv.ylabel('Beta')
ipv.ylim(0,pi)
ipv.zlabel('E')
ipv.zlim(2,6)
ipv.show()
